In [2]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

df = pd.read_csv('Data/Dataset.csv', decimal =".", thousands=",")
df = df.drop(['Kota','Paslon 1','Paslon 2','Total'],axis=1)

scaler = MinMaxScaler()

df_minmax = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
dfX, dfY = df_minmax.iloc[:, :-1], df_minmax.iloc[:, [-1]]
kf = RepeatedKFold(n_splits=10, n_repeats = 3 , random_state=1)
kf.get_n_splits(dfX)
y = df['Partisipasi']


def beep():
    import IPython
    IPython.display.Audio("beep.mp3", autoplay=True)

def flatten(l):
    return [item for sublist in l for item in sublist]

In [7]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils

def objective(trial):
    data, target = dfX, y
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
    train_pool = Pool(train_x, train_y)
    val_pool = Pool(val_x, val_y)
    test_pool = Pool(test_x)

    params = {
        "objective": trial.suggest_categorical("objective", ["RMSE"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
         "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    model = CatBoostRegressor(**params)

    pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
    model.fit(train_pool, eval_set=val_pool, callbacks=[pruning_callback], verbose=500, early_stopping_rounds=100)

    pruning_callback.check_pruned()

    preds = model.predict(test_pool)

    results = utils.error_rate("Catboost",preds,test_y)
    print(results)

    rmse = mean_squared_error(test_y, preds, squared=False)
    return rmse


study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))

study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


    


    






[I 2023-06-07 20:12:20,347] A new study created in memory with name: no-name-1f5096c1-7af5-48ed-a82d-ebf1bc8438f1
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\3008741279.py:33: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


Learning rate set to 0.032991
0:	learn: 0.0480222	test: 0.0526873	best: 0.0526873 (0)	total: 2.35ms	remaining: 2.35s
500:	learn: 0.0352131	test: 0.0473180	best: 0.0473180 (500)	total: 579ms	remaining: 576ms


[I 2023-06-07 20:12:22,229] Trial 0 finished with value: 0.0349815838846568 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.019294785961930627, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.6044011355658085}. Best is trial 0 with value: 0.0349815838846568.


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04686541213
bestIteration = 596

Shrink model to first 597 iterations.
('Catboost', 0.0349815838846568, 0.027204473841132337, 0.4295501324368346)
Learning rate set to 0.032991
0:	learn: 0.0491757	test: 0.0445211	best: 0.0445211 (0)	total: 249us	remaining: 249ms


C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\3008741279.py:33: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


...

0:	learn: 0.0510492	test: 0.0538492	best: 0.0538492 (0)	total: 242us	remaining: 243ms

bestTest = 0.05277874674
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0489718	test: 0.0520263	best: 0.0520263 (0)	total: 281us	remaining: 282ms

bestTest = 0.05202625615
bestIteration = 0

Shrink model to first 1 iterations.
Learning rate set to 0.032991
0:	learn: 0.0458405	test: 0.0536157	best: 0.0536157 (0)	total: 240us	remaining: 240ms

bestTest = 0.05361573074
bestIteration = 1

Shrink model to first 2 iterations.


[I 2023-06-07 20:12:46,878] Trial 97 pruned. Trial was pruned at iteration 10.
[I 2023-06-07 20:12:46,997] Trial 98 pruned. Trial was pruned at iteration 10.


Learning rate set to 0.032991
0:	learn: 0.0475042	test: 0.0443993	best: 0.0443993 (0)	total: 5.14ms	remaining: 5.14s

bestTest = 0.04325713696
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0550404	test: 0.0359751	best: 0.0359751 (0)	total: 270us	remaining: 271ms


[I 2023-06-07 20:12:47,161] Trial 99 pruned. Trial was pruned at iteration 95.



bestTest = 0.03364072166
bestIteration = 89

Shrink model to first 90 iterations.
Number of finished trials: 100
Best trial:
  Value: 0.0349815838846568
  Params: 
    objective: RMSE
    colsample_bylevel: 0.019294785961930627
    depth: 6
    boosting_type: Ordered
    bootstrap_type: Bayesian
    bagging_temperature: 4.6044011355658085


In [8]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils

def objective(trial):
    cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
    data, target = dfX[cols], y
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
    train_pool = Pool(train_x, train_y)
    val_pool = Pool(val_x, val_y)
    test_pool = Pool(test_x)

    params = {
        "objective": trial.suggest_categorical("objective", ["RMSE"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
         "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    model = CatBoostRegressor(**params)

    pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
    model.fit(train_pool, eval_set=val_pool, callbacks=[pruning_callback], verbose=500, early_stopping_rounds=100)

    pruning_callback.check_pruned()

    preds = model.predict(test_pool)

    results = utils.error_rate("Catboost",preds,test_y)
    print(results)

    rmse = mean_squared_error(test_y, preds, squared=False)
    return rmse



study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))

study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


    


    






[I 2023-06-07 20:14:59,108] A new study created in memory with name: no-name-e719ca62-4c82-45dd-8d65-6e076a7b18e0
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


Learning rate set to 0.032991
0:	learn: 0.0486389	test: 0.0492778	best: 0.0492778 (0)	total: 977us	remaining: 976ms
500:	learn: 0.0322777	test: 0.0391103	best: 0.0390906 (491)	total: 518ms	remaining: 516ms


[I 2023-06-07 20:15:02,012] Trial 0 finished with value: 0.036636711002973305 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.052503740365612624, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.036636711002973305.


999:	learn: 0.0268657	test: 0.0361930	best: 0.0361658 (958)	total: 1.06s	remaining: 0us

bestTest = 0.03616579889
bestIteration = 958

Shrink model to first 959 iterations.
('Catboost', 0.036636711002973305, 0.028105386558936983, 0.40950852705839)
Learning rate set to 0.032991
0:	learn: 0.0453744	test: 0.0597449	best: 0.0597449 (0)	total: 267us	remaining: 267ms


C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


...

Learning rate set to 0.032991
0:	learn: 0.0484320	test: 0.0412963	best: 0.0412963 (0)	total: 1.28ms	remaining: 1.28s

bestTest = 0.04119179183
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0492385	test: 0.0512994	best: 0.0512994 (0)	total: 308us	remaining: 309ms

bestTest = 0.0506999177
bestIteration = 6

Shrink model to first 7 iterations.
Learning rate set to 0.032991
0:	learn: 0.0496944	test: 0.0426903	best: 0.0426903 (0)	total: 1.28ms	remaining: 1.28s


[I 2023-06-07 20:15:47,810] Trial 96 pruned. Trial was pruned at iteration 10.
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
[I 2023-06-07 20:15:47,893] Trial 97 pruned. Trial was pruned at iteration 10.
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
[I 2023-06-07 20:15:48,001] Trial 98 pruned. Trial was pruned at iteration 10.



bestTest = 0.04199504562
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0455415	test: 0.0541023	best: 0.0541023 (0)	total: 302us	remaining: 302ms

bestTest = 0.05316446872
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0444556	test: 0.0510119	best: 0.0510119 (0)	total: 1.15ms	remaining: 1.15s

bestTest = 0.0509160874
bestIteration = 6

Shrink model to first 7 iterations.


C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
[I 2023-06-07 20:15:48,105] Trial 99 pruned. Trial was pruned at iteration 10.


Learning rate set to 0.032991
0:	learn: 0.0480794	test: 0.0538179	best: 0.0538179 (0)	total: 1.21ms	remaining: 1.21s

bestTest = 0.05374895923
bestIteration = 8

Shrink model to first 9 iterations.
Number of finished trials: 100
Best trial:
  Value: 0.036636711002973305
  Params: 
    objective: RMSE
    colsample_bylevel: 0.052503740365612624
    depth: 9
    boosting_type: Ordered
    bootstrap_type: MVS


In [9]:
cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
utils.evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.03512429844587696, 0.028615520062230004, 0.47568209883632273)

In [11]:


def evaluate_catboost(dfX,y,kf):
  from catboost import CatBoostRegressor
  from catboost import Pool
  from sklearn.metrics import r2_score, mean_squared_error
  vals = []
  preds = []
  error_rates_dt = []

  cbr = CatBoostRegressor(
    boosting_type= 'Ordered',
    colsample_bylevel= 0.052503740365612624,
    depth= 9,
    bootstrap_type= 'MVS',
    )
  for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      train_pool = Pool(data=X_train, label=y_train)
      test_pool = Pool(data=X_test, label=y_test)

      cbr.fit(train_pool, verbose=False)
      cbr_pred = cbr.predict(test_pool)
      preds.append(cbr_pred)
      vals.append(y_test)

  cat_df = pd.DataFrame({'Prediction':utils.flatten(preds),'Actual Values': utils.flatten(vals)})
  return utils.error_rate("Catboost",cat_df['Prediction'],cat_df['Actual Values'])

evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.0379735377563578, 0.030907234780759615, 0.3871679255729855)

Shrink model to first 90 iterations.
Number of finished trials: 100
Best trial:
  Value: 0.0349815838846568
  Params: 
    objective: RMSE
    colsample_bylevel: 0.019294785961930627
    depth: 6
    boosting_type: Ordered
    bootstrap_type: Bayesian
    bagging_temperature: 4.6044011355658085

In [12]:


def evaluate_catboost(dfX,y,kf):
  from catboost import CatBoostRegressor
  from catboost import Pool
  from sklearn.metrics import r2_score, mean_squared_error
  vals = []
  preds = []
  error_rates_dt = []

  cbr = CatBoostRegressor(
    boosting_type= 'Ordered',
    colsample_bylevel= 0.019294785961930627,
    depth= 6,
    bootstrap_type= 'Bayesian',
    bagging_temperature= 4.6044011355658085,
    )
  for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      train_pool = Pool(data=X_train, label=y_train)
      test_pool = Pool(data=X_test, label=y_test)

      cbr.fit(train_pool, verbose=False)
      cbr_pred = cbr.predict(test_pool)
      preds.append(cbr_pred)
      vals.append(y_test)

  cat_df = pd.DataFrame({'Prediction':utils.flatten(preds),'Actual Values': utils.flatten(vals)})
  return utils.error_rate("Catboost",cat_df['Prediction'],cat_df['Actual Values'])

evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.04339805479488252, 0.03453902698156212, 0.19957637991715627)

In [14]:
model = CatBoostRegressor(verbose=1000)
grid = {
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'depth': [4, 6, 10, 12],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
grid_search_result = model.grid_search(grid, X=dfX[cols], y=y, plot=True, cv=kf, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7794596	test: 0.8047215	best: 0.8047215 (0)	total: 704us	remaining: 704ms
999:	learn: 0.0017389	test: 0.0857531	best: 0.0857467 (985)	total: 600ms	remaining: 0us

bestTest = 0.08574666853
bestIteration = 985

0:	learn: 0.7657917	test: 0.7909031	best: 0.7909031 (0)	total: 519us	remaining: 519ms
999:	learn: 0.0004218	test: 0.0934371	best: 0.0934209 (979)	total: 753ms	remaining: 0us

bestTest = 0.09342090662
bestIteration = 979

0:	learn: 0.7521271	test: 0.7770863	best: 0.7770863 (0)	total: 654us	remaining: 654ms
999:	learn: 0.0000640	test: 0.0936871	best: 0.0936867 (995)	total: 688ms	remaining: 0us

bestTest = 0.09368672517
bestIteration = 995

0:	learn: 0.7316370	test: 0.7563645	best: 0.7563645 (0)	total: 476us	remaining: 476ms
999:	learn: 0.0000073	test: 0.0916838	best: 0.0916838 (999)	total: 648ms	remaining: 0us

bestTest = 0.0916838414
bestIteration = 999

0:	learn: 0.7836079	test: 0.8090148	best: 0.8090148 (0)	total: 781us	remaining: 781ms
999:	learn: 0.0037578	test: 0.0

In [17]:
grid_search_result['params']

{'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.03}

In [18]:


def evaluate_catboost(dfX,y,kf):
  from catboost import CatBoostRegressor
  from catboost import Pool
  from sklearn.metrics import r2_score, mean_squared_error
  vals = []
  preds = []
  error_rates_dt = []

  cbr = CatBoostRegressor(
    depth= 4,
    l2_leaf_reg= 1,
    learning_rate= 0.03,
    )
  for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      train_pool = Pool(data=X_train, label=y_train)
      test_pool = Pool(data=X_test, label=y_test)

      cbr.fit(train_pool, verbose=False)
      cbr_pred = cbr.predict(test_pool)
      preds.append(cbr_pred)
      vals.append(y_test)

  cat_df = pd.DataFrame({'Prediction':utils.flatten(preds),'Actual Values': utils.flatten(vals)})
  return utils.error_rate("Catboost",cat_df['Prediction'],cat_df['Actual Values'])

evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.03603586259414829, 0.029358332918660912, 0.4481142088724709)

In [7]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
import Utils as utils
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from optuna.integration import CatBoostPruningCallback


def objective(trial):
    cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
    usedDF = dfX[cols]
    # data, target = dfX[cols], y
    # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    # train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
    # train_pool = Pool(train_x, train_y)
    # val_pool = Pool(val_x, val_y)
    # test_pool = Pool(test_x)

    preds = []
    vals = []

    params = {
        "objective": trial.suggest_categorical("objective", ["RMSE"]),
        "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.001, 0.05, 0.005),
        "depth": trial.suggest_int("depth", 8, 16),
        "l2_leaf_reg": trial.suggest_discrete_uniform("l2_leaf_reg", 1, 5.5, 0.5),
        "use_best_model": True,
        "random_seed": 0,
    }

    print(params)

    # if params["bootstrap_type"] == "Bayesian":
    #     params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    # elif params["bootstrap_type"] == "Bernoulli":
    #     params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    model = CatBoostRegressor(**params, verbose=False)

    # pruning_callback = CatBoostPruningCallback(trial, "RMSE")

    for i, (train_index , test_index) in enumerate(kf.split(usedDF)):
        train_x, test_x = usedDF.iloc[train_index], usedDF.iloc[test_index]
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
        train_pool = Pool(train_x, train_y)
        val_pool = Pool(val_x, val_y)
        test_pool = Pool(test_x)

        model.fit(train_pool, eval_set=val_pool)

        pred = model.predict(test_pool)
        # convert np float array to list

        preds.append(pred.tolist())
        vals.append(test_y.values)
        
    
    preds = flatten(preds)
    vals = flatten(vals)
    
    results = utils.error_rate("Catboost",preds ,vals)
    print(results)

    rmse = mean_squared_error(vals, preds, squared=False)
    return rmse



study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))

study.optimize(objective, n_trials=15)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))




[I 2023-06-08 14:38:08,042] A new study created in memory with name: no-name-a820b070-628f-4db5-9c86-d7caba9e046e
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_26256\3613356961.py:26: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.001, 0.05, 0.005),
c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\distributions.py:685: UserWarning: The distribution is specified by [0.001, 0.05] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.001, 0.046].
  warnings.warn(
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_26256\3613356961.py:28: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com

{'objective': 'RMSE', 'learning_rate': 0.001, 'depth': 16, 'l2_leaf_reg': 2.0, 'use_best_model': True, 'random_seed': 0}


[W 2023-06-08 14:38:28,381] Trial 0 failed with parameters: {'objective': 'RMSE', 'learning_rate': 0.001, 'depth': 16, 'l2_leaf_reg': 2.0} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_26256\3613356961.py", line 53, in objective
    model.fit(train_pool, eval_set=val_pool)
  File "c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 2355, in _fit
    self._train(
  File "c:\Users\Bayu 

KeyboardInterrupt: 

In [43]:
from winsound import Beep

Beep(300, 200)
Beep(300, 200)
Beep(300, 700)

In [8]:
import Utils as utils
from catboost import CatBoostRegressor
from catboost import Pool

# utils.evaluate_catboost(dfX,y,kf)
def cbpred(dfX,y,kf):
    vals = []
    preds = []
    cbr = CatBoostRegressor()
    for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        train_pool = Pool(data=X_train, label=y_train)
        test_pool = Pool(data=X_test, label=y_test)

        
        cbr.fit(train_pool, verbose=False)
        cbr_pred = cbr.predict(test_pool)
        print(cbr.get_all_params())
        preds.append(cbr_pred)
        vals.append(y_test)

    return utils.error_rate("Catboost",flatten(preds),flatten(vals))
cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']

res = cbpred(dfX[cols],y,kf)


{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 1, 'use_best_model': False, 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.028226999565958977, 'score_function': 'Cosine', 'task_type': 'CPU', 'leaf_estimation_ite

In [9]:
res

('Catboost', 0.03512429844587696, 0.028615520062230004, 0.47568209883632273)

In [21]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils

def objective(trial):
    data, target = dfX, y
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    train_pool = Pool(train_x, train_y)
    test_pool = Pool(test_x)

    model = CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    model.fit(train_pool, verbose=False)
    preds = model.predict(test_pool)
    results = utils.error_rate("Catboost",preds,test_y)
    print(results)

    rmse = mean_squared_error(test_y, preds, squared=False)
    return rmse


sampler = optuna.samplers.TPESampler(seed=1)
study = optuna.create_study(direction="minimize",sampler = sampler)

study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2023-06-22 02:15:24,556] A new study created in memory with name: no-name-ca317c89-a8d6-42e5-97a8-fdf3f082beff
[I 2023-06-22 02:15:25,105] Trial 0 finished with value: 0.04802717139882986 and parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.04802717139882986.


('Catboost', 0.04802717139882986, 0.04093203093536998, 0.28182475257848016)


[I 2023-06-22 02:15:27,655] Trial 1 finished with value: 0.03572594177875271 and parameters: {'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32}. Best is trial 1 with value: 0.03572594177875271.


('Catboost', 0.03572594177875271, 0.02950171591270989, 0.34446362363544236)


[I 2023-06-22 02:15:29,366] Trial 2 finished with value: 0.04356138008906657 and parameters: {'iterations': 226, 'learning_rate': 0.002490020818620744, 'depth': 9, 'l2_leaf_reg': 48.15215820395775, 'bootstrap_type': 'Bayesian', 'random_strength': 6.6193844201488494e-06, 'bagging_temperature': 6.923226156693141, 'od_type': 'Iter', 'od_wait': 13}. Best is trial 1 with value: 0.03572594177875271.


...

('Catboost', 0.04299583529698311, 0.034667601339443695, -0.015807816665741825)


[I 2023-06-22 02:21:58,754] Trial 98 finished with value: 0.040541823052823965 and parameters: {'iterations': 663, 'learning_rate': 0.03832357216840194, 'depth': 7, 'l2_leaf_reg': 1.1115851837700547e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 6.18867125766955, 'bagging_temperature': 2.084447222058218, 'od_type': 'IncToDec', 'od_wait': 47}. Best is trial 58 with value: 0.026437968396982793.


('Catboost', 0.040541823052823965, 0.03346328246209946, -0.10494767542652728)


[I 2023-06-22 02:22:12,676] Trial 99 finished with value: 0.03528062773874751 and parameters: {'iterations': 836, 'learning_rate': 0.08028344635340928, 'depth': 10, 'l2_leaf_reg': 4.217323518217559e-07, 'bootstrap_type': 'Bayesian', 'random_strength': 0.28424185630338955, 'bagging_temperature': 2.563476436097984, 'od_type': 'IncToDec', 'od_wait': 45}. Best is trial 58 with value: 0.026437968396982793.


('Catboost', 0.03528062773874751, 0.028349582676150155, 0.2011040176230674)
Number of finished trials: 100
Best trial:
  Value: 0.026437968396982793
  Params: 
    iterations: 843
    learning_rate: 0.03549668630252779
    depth: 10
    l2_leaf_reg: 2.7702750401932485e-06
    bootstrap_type: Bayesian
    random_strength: 1.989872891341923
    bagging_temperature: 0.8515228814732063
    od_type: IncToDec
    od_wait: 44


In [23]:
import Utils as utils
from catboost import CatBoostRegressor
from catboost import Pool

# utils.evaluate_catboost(dfX,y,kf)
def cbpred_params(dfX,y,kf, params):
    vals = []
    preds = []
    cbr = CatBoostRegressor(**params)
    for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
        train_x, train_y = dfX.iloc[train_index], y.iloc[train_index]
        # train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
        test_x, test_y = dfX.iloc[test_index], y.iloc[test_index]

        train_pool = Pool(data=train_x, label=train_y)
        # val_pool = Pool(data=val_x, label=val_y)
        test_pool = Pool(data=test_x, label=test_y)

        
        cbr.fit(train_pool, verbose=False)
        cbr_pred = cbr.predict(test_pool)
        print(cbr.get_all_params())
        preds.append(cbr_pred)
        vals.append(test_y)

    return utils.error_rate("Catboost",flatten(preds),flatten(vals))


cols = ['Umur', 'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
params = trial.params

res = cbpred_params(dfX[cols],y,kf, params)



{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 843, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'od_pval': 0, 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 2.7702749321179e-06, 'random_strength': 1.989872932434082, 'od_type': 'IncToDec', 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'use_best_model': False, 'od_wait': 44, 'random_seed': 0, 'depth': 10, 'posterior_sampling': False, 'border_count': 254, 'bagging_temperature': 0.8515228629112244, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'R

In [22]:
trial.params

{'iterations': 843,
 'learning_rate': 0.03549668630252779,
 'depth': 10,
 'l2_leaf_reg': 2.7702750401932485e-06,
 'bootstrap_type': 'Bayesian',
 'random_strength': 1.989872891341923,
 'bagging_temperature': 0.8515228814732063,
 'od_type': 'IncToDec',
 'od_wait': 44}

In [24]:
res

('Catboost', 0.03515836949999622, 0.028029121148614687, 0.474664414294787)

In [ ]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

df = pd.read_csv('Data/Dataset.csv', decimal =".", thousands=",")
df = df.drop(['Kota','Paslon 1','Paslon 2','Total'],axis=1)

scaler = MinMaxScaler()

df_minmax = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
dfX, dfY = df_minmax.iloc[:, :-1], df_minmax.iloc[:, [-1]]
kf = RepeatedKFold(n_splits=10, n_repeats = 3 , random_state=1)
kf.get_n_splits(dfX)
y = df['Partisipasi']


def beep():
    import IPython
    IPython.display.Audio("beep.mp3", autoplay=True)

def flatten(l):
    return [item for sublist in l for item in sublist]

In [25]:
cols = ['Umur', 'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
dfX = dfX[cols]
dfX

,Umur,Umur Squared,APM SMA,SMA,Pengeluaran Bulanan,Indeks Pembangunan Manusia,DPT,Sentiment,Weighted Sentiment
0,0.271381,0.228896,0.719929,0.000000,0.297269,0.414730,0.000000,0.417106,0.50412
1,0.388596,0.337547,0.431031,0.648352,0.935966,0.844805,0.497812,0.417106,0.50412
2,0.414311,0.362173,0.625690,0.670330,0.819952,0.909689,0.644910,0.417106,0.50412
3,0.557591,0.504589,0.492827,0.285714,0.943144,0.846120,0.229025,0.417106,0.50412
4,0.406859,0.355008,0.664092,0.483516,0.804669,0.799211,0.357522,0.417106,0.50412
...,...,...,...,...,...,...,...,...,...
101,0.450972,0.397774,0.548223,0.054945,0.287647,0.583516,0.037204,1.000000,1.00000
102,0.564808,0.511995,0.933569,0.054945,0.413812,0.702324,0.022921,1.000000,1.00000
103,0.745254,0.704463,0.957625,0.060440,0.497786,0.830338,0.037216,1.000000,1.00000
104,0.490701,0.437005,0.614434,0.763736,0.862860,0.889084,0.611749,1.000000,1.00000


In [26]:
import Utils as utils
from catboost import CatBoostRegressor
from catboost import Pool

# utils.evaluate_catboost(dfX,y,kf)
def cbpred(dfX,y,kf):
    vals = []
    preds = []
    cbr = CatBoostRegressor()
    for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        train_pool = Pool(data=X_train, label=y_train)
        test_pool = Pool(data=X_test, label=y_test)

        
        cbr.fit(train_pool, verbose=False)
        cbr_pred = cbr.predict(test_pool)
        print(cbr.get_all_params())
        preds.append(cbr_pred)
        vals.append(y_test)

    return utils.error_rate("Catboost",flatten(preds),flatten(vals))

res = cbpred(dfX,y,kf)
res


{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 1, 'use_best_model': False, 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.028226999565958977, 'score_function': 'Cosine', 'task_type': 'CPU', 'leaf_estimation_ite

('Catboost', 0.03548927382169304, 0.029194853794929546, 0.4647291485798465)

In [36]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils
import math

def objective(trial):
    model = CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    preds = []
    vals = []
    for train_index, test_index in kf.split(dfX):
        train_x, train_y = dfX.iloc[train_index], y.iloc[train_index]
        test_x, test_y = dfX.iloc[test_index], y.iloc[test_index]

        train_pool = Pool(data=train_x, label=train_y)
        test_pool = Pool(data=test_x, label=test_y)

        model.fit(train_pool, verbose=False)
        pred = model.predict(test_pool)
        preds.append(pred)
        vals.append(test_y) 

    results = utils.error_rate("Catboost",flatten(preds),flatten(vals))
    print(results)
    rmse = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))
    return rmse


sampler = optuna.samplers.TPESampler(seed=1)
study = optuna.create_study(direction="minimize",sampler = sampler)

study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2023-06-22 03:11:47,311] A new study created in memory with name: no-name-4e916f32-61ca-4a48-8e5f-fd2d4f899eed


[I 2023-06-22 03:11:59,050] Trial 0 finished with value: 0.03642721476769824 and parameters: {'iterations': 475, 'learning_rate': 0.027583475549166746, 'depth': 4, 'l2_leaf_reg': 1.0551779964424746e-05, 'bootstrap_type': 'Bayesian', 'random_strength': 2.0931628460945333e-07, 'bagging_temperature': 0.923385947687978, 'od_type': 'Iter', 'od_wait': 26}. Best is trial 0 with value: 0.03642721476769824.


('Catboost', 0.03642721476769824, 0.030143808240450282, 0.43606207652223017)


[I 2023-06-22 03:12:37,124] Trial 1 finished with value: 0.03595156429103307 and parameters: {'iterations': 585, 'learning_rate': 0.006892694481137703, 'depth': 8, 'l2_leaf_reg': 1.10795595820296e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 0.7999391045172093, 'bagging_temperature': 0.27387593197926163, 'od_type': 'IncToDec', 'od_wait': 32}. Best is trial 1 with value: 0.03595156429103307.


('Catboost', 0.03595156429103307, 0.02903872046061371, 0.4506932297113502)


...

('Catboost', 0.03427073508642016, 0.02808703038149281, 0.5008555942214159)


[I 2023-06-22 03:52:39,336] Trial 98 finished with value: 0.03452062979556547 and parameters: {'iterations': 934, 'learning_rate': 0.01275059086234731, 'depth': 4, 'l2_leaf_reg': 6.015401452019411e-07, 'bootstrap_type': 'Bayesian', 'random_strength': 1.0237013232054633, 'bagging_temperature': 3.3446387138489238, 'od_type': 'Iter', 'od_wait': 38}. Best is trial 67 with value: 0.03408314614979994.


('Catboost', 0.03452062979556547, 0.028333106015495615, 0.49354975087347175)


[I 2023-06-22 03:52:58,337] Trial 99 finished with value: 0.034574237847082905 and parameters: {'iterations': 881, 'learning_rate': 0.007672273916660241, 'depth': 4, 'l2_leaf_reg': 5.304977891363944e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 2.3497732669062725, 'bagging_temperature': 5.4370529755576165, 'od_type': 'Iter', 'od_wait': 45}. Best is trial 67 with value: 0.03408314614979994.


('Catboost', 0.034574237847082905, 0.02824053501168026, 0.4919755680119443)
Number of finished trials: 100
Best trial:
  Value: 0.03408314614979994
  Params: 
    iterations: 836
    learning_rate: 0.018369975117039684
    depth: 5
    l2_leaf_reg: 1.1994924961669811e-06
    bootstrap_type: Bayesian
    random_strength: 9.311341227775438
    bagging_temperature: 4.088561104847901
    od_type: Iter
    od_wait: 44


In [37]:
import Utils as utils
from catboost import CatBoostRegressor
from catboost import Pool

# utils.evaluate_catboost(dfX,y,kf)
def cbpred_params(dfX,y,kf, params):
    vals = []
    preds = []
    cbr = CatBoostRegressor(**params)
    for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
        train_x, train_y = dfX.iloc[train_index], y.iloc[train_index]
        # train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
        test_x, test_y = dfX.iloc[test_index], y.iloc[test_index]

        train_pool = Pool(data=train_x, label=train_y)
        # val_pool = Pool(data=val_x, label=val_y)
        test_pool = Pool(data=test_x, label=test_y)

        
        cbr.fit(train_pool, verbose=False)
        cbr_pred = cbr.predict(test_pool)
        print(cbr.get_all_params())
        preds.append(cbr_pred)
        vals.append(test_y)

    return utils.error_rate("Catboost",flatten(preds),flatten(vals))


cols = ['Umur', 'Umur Squared', 'APM SMA', 'SMA', 'Pengeluaran Bulanan', 'Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
params = trial.params
# params = {'iterations': 843,
#  'learning_rate': 0.03549668630252779,
#  'depth': 10,
#  'l2_leaf_reg': 2.7702750401932485e-06,
#  'bootstrap_type': 'Bayesian',
#  'random_strength': 1.989872891341923,
#  'bagging_temperature': 0.8515228814732063,
#  'od_type': 'IncToDec',
#  'od_wait': 44}


res = cbpred_params(dfX[cols],y,kf, params)



{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 836, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'od_pval': 0, 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 1.199492544401437e-06, 'random_strength': 9.311341285705566, 'od_type': 'Iter', 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'use_best_model': False, 'od_wait': 44, 'random_seed': 0, 'depth': 5, 'posterior_sampling': False, 'border_count': 254, 'bagging_temperature': 4.088561058044434, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE'

In [39]:
currbest = trial.params
currbest

{'iterations': 836,
 'learning_rate': 0.018369975117039684,
 'depth': 5,
 'l2_leaf_reg': 1.1994924961669811e-06,
 'bootstrap_type': 'Bayesian',
 'random_strength': 9.311341227775438,
 'bagging_temperature': 4.088561104847901,
 'od_type': 'Iter',
 'od_wait': 44}

In [38]:
res

('Catboost', 0.03408314614979994, 0.027608649962985995, 0.5063050077971629)

In [42]:
trialdf = study.trials_dataframe(multi_index=True)
trialdf

number     value             datetime_start          datetime_complete   
                                                                            
0       0  0.036427 2023-06-22 03:11:47.313552 2023-06-22 03:11:59.050418  \
1       1  0.035952 2023-06-22 03:11:59.052115 2023-06-22 03:12:37.124578   
2       2  0.048229 2023-06-22 03:12:37.126489 2023-06-22 03:13:01.674536   
3       3  0.045082 2023-06-22 03:13:01.676113 2023-06-22 03:13:31.687110   
4       4  0.034440 2023-06-22 03:13:31.688616 2023-06-22 03:13:46.360778   
..    ...       ...                        ...                        ...   
95     95  0.034645 2023-06-22 03:51:21.537533 2023-06-22 03:51:42.157417   
96     96  0.034213 2023-06-22 03:51:42.158416 2023-06-22 03:52:02.131600   
97     97  0.034271 2023-06-22 03:52:02.133191 2023-06-22 03:52:19.870155   
98     98  0.034521 2023-06-22 03:52:19.871155 2023-06-22 03:52:39.336154   
99     99  0.034574 2023-06-22 03:52:39.337699 2023-06-22 03:52:58.337164   

                 duration              params                                   
                          bagging_temperature bootstrap_type depth iterations   
0  0 days 00:00:11.736866            0.923386       Bayesian     4        475  \
1  0 days 00:00:38.072463            0.273876       Bayesian     8        585   
2  0 days 00:00:24.548047            6.923226       Bayesian     9        226   
3  0 days 00:00:30.010997            9.578895       Bayesian    10        135   
4  0 days 00:00:14.672162            7.481657       Bayesian     4        718   
..                    ...                 ...            ...   ...        ...   
95 0 days 00:00:20.619884            3.566821       Bayesian     4        955   
96 0 days 00:00:19.973184            4.825321       Bayesian     4        935   
97 0 days 00:00:17.736964            4.833429       Bayesian     4        900   
98 0 days 00:00:19.464999            3.344639       Bayesian     4        934   
99 0 days 00:00:18.999465            5.437053       Bayesian     4        881   

                                                                     state  
     l2_leaf_reg learning_rate   od_type od_wait random_strength            
0   1.055178e-05      0.027583      Iter      26    2.093163e-07  COMPLETE  
1   1.107956e-06      0.006893  IncToDec      32    7.999391e-01  COMPLETE  
2   4.815216e+01      0.002490      Iter      13    6.619384e-06  COMPLETE  
3   9.626498e-08      0.002186      Iter      22    6.165505e-05  COMPLETE  
4   3.172803e-01      0.046693      Iter      14    7.938707e+00  COMPLETE  
..           ...           ...       ...     ...             ...       ...  
95  1.305424e-07      0.013921      Iter      43    1.524109e+00  COMPLETE  
96  2.302908e-06      0.008837      Iter      42    6.515164e+00  COMPLETE  
97  2.317256e-06      0.009129      Iter      44    6.601977e+00  COMPLETE  
98  6.015401e-07      0.012751      Iter      38    1.023701e+00  COMPLETE  
99  5.304978e-06      0.007672      Iter      45    2.349773e+00  COMPLETE  

[100 rows x 15 columns]

In [43]:
trialdf['number'] = trialdf['number'] + 1
trialdf

number     value             datetime_start          datetime_complete   
                                                                            
0       1  0.036427 2023-06-22 03:11:47.313552 2023-06-22 03:11:59.050418  \
1       2  0.035952 2023-06-22 03:11:59.052115 2023-06-22 03:12:37.124578   
2       3  0.048229 2023-06-22 03:12:37.126489 2023-06-22 03:13:01.674536   
3       4  0.045082 2023-06-22 03:13:01.676113 2023-06-22 03:13:31.687110   
4       5  0.034440 2023-06-22 03:13:31.688616 2023-06-22 03:13:46.360778   
..    ...       ...                        ...                        ...   
95     96  0.034645 2023-06-22 03:51:21.537533 2023-06-22 03:51:42.157417   
96     97  0.034213 2023-06-22 03:51:42.158416 2023-06-22 03:52:02.131600   
97     98  0.034271 2023-06-22 03:52:02.133191 2023-06-22 03:52:19.870155   
98     99  0.034521 2023-06-22 03:52:19.871155 2023-06-22 03:52:39.336154   
99    100  0.034574 2023-06-22 03:52:39.337699 2023-06-22 03:52:58.337164   

                 duration              params                                   
                          bagging_temperature bootstrap_type depth iterations   
0  0 days 00:00:11.736866            0.923386       Bayesian     4        475  \
1  0 days 00:00:38.072463            0.273876       Bayesian     8        585   
2  0 days 00:00:24.548047            6.923226       Bayesian     9        226   
3  0 days 00:00:30.010997            9.578895       Bayesian    10        135   
4  0 days 00:00:14.672162            7.481657       Bayesian     4        718   
..                    ...                 ...            ...   ...        ...   
95 0 days 00:00:20.619884            3.566821       Bayesian     4        955   
96 0 days 00:00:19.973184            4.825321       Bayesian     4        935   
97 0 days 00:00:17.736964            4.833429       Bayesian     4        900   
98 0 days 00:00:19.464999            3.344639       Bayesian     4        934   
99 0 days 00:00:18.999465            5.437053       Bayesian     4        881   

                                                                     state  
     l2_leaf_reg learning_rate   od_type od_wait random_strength            
0   1.055178e-05      0.027583      Iter      26    2.093163e-07  COMPLETE  
1   1.107956e-06      0.006893  IncToDec      32    7.999391e-01  COMPLETE  
2   4.815216e+01      0.002490      Iter      13    6.619384e-06  COMPLETE  
3   9.626498e-08      0.002186      Iter      22    6.165505e-05  COMPLETE  
4   3.172803e-01      0.046693      Iter      14    7.938707e+00  COMPLETE  
..           ...           ...       ...     ...             ...       ...  
95  1.305424e-07      0.013921      Iter      43    1.524109e+00  COMPLETE  
96  2.302908e-06      0.008837      Iter      42    6.515164e+00  COMPLETE  
97  2.317256e-06      0.009129      Iter      44    6.601977e+00  COMPLETE  
98  6.015401e-07      0.012751      Iter      38    1.023701e+00  COMPLETE  
99  5.304978e-06      0.007672      Iter      45    2.349773e+00  COMPLETE  

[100 rows x 15 columns]

In [44]:
trialdf.to_excel('optuna.xlsx')


In [46]:
model = CatBoostRegressor(verbose=1000)
grid = {
    'learning_rate': [0.017, 0.018, 0.019],
    'depth': [4 , 5],
    'l2_leaf_reg': [1, 1.2, 1.4],
    'random_strength': [9,2, 9.4, 9.6],
    'bagging_temperature': [4.088561104847901],
    'od_wait': [44],
    'od_type': ['Iter'],
    'bootstrap_type': ['Bayesian'],

}
grid_search_result = model.grid_search(grid, X=dfX[cols], y=y, plot=True, cv=kf, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7877144	test: 0.8139871	best: 0.8139871 (0)	total: 615us	remaining: 615ms
999:	learn: 0.0111392	test: 0.0823373	best: 0.0823373 (999)	total: 765ms	remaining: 0us

bestTest = 0.08233734489
bestIteration = 999

0:	learn: 0.7869937	test: 0.8133130	best: 0.8133130 (0)	total: 512us	remaining: 512ms
999:	learn: 0.0104203	test: 0.0815170	best: 0.0815170 (999)	total: 731ms	remaining: 0us

bestTest = 0.08151701493
bestIteration = 999

0:	learn: 0.7862731	test: 0.8126389	best: 0.8126389 (0)	total: 1.18ms	remaining: 1.17s
999:	learn: 0.0096602	test: 0.0810711	best: 0.0810711 (999)	total: 652ms	remaining: 0us

bestTest = 0.0810711163
bestIteration = 999

0:	learn: 0.7883816	test: 0.8156402	best: 0.8156402 (0)	total: 787us	remaining: 787ms
Stopped by overfitting detector  (44 iterations wait)

bestTest = 0.1049830917
bestIteration = 954

0:	learn: 0.7877002	test: 0.8150634	best: 0.8150634 (0)	total: 732us	remaining: 731ms
999:	learn: 0.0058523	test: 0.1052790	best: 0.1052673 (975)	total

In [47]:
grid_search_result['params']

{'depth': 4,
 'random_strength': 2,
 'od_wait': 44,
 'l2_leaf_reg': 1,
 'bagging_temperature': 4.088561104847901,
 'learning_rate': 0.018,
 'od_type': 'Iter',
 'bootstrap_type': 'Bayesian'}

In [ ]:
{'iterations': 836,
 'learning_rate': 0.018369975117039684,
 'depth': 5,
 'l2_leaf_reg': 1.1994924961669811e-06,
 'bootstrap_type': 'Bayesian',
 'random_strength': 9.311341227775438,
 'bagging_temperature': 4.088561104847901,
 'od_type': 'Iter',
 'od_wait': 44}


In [50]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils
import math

def objective(trial):
    model = CatBoostRegressor(
        learning_rate=trial.suggest_float("learning_rate", 1e-2, 3e-2, log=True),
        depth=trial.suggest_int("depth", 4, 6),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 2e-06, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 3, 5.0),
        od_type=trial.suggest_categorical("od_type", ["Iter"]),
        od_wait=trial.suggest_int("od_wait", 40, 50),
        verbose=False
    )
    preds = []
    vals = []
    for train_index, test_index in kf.split(dfX):
        train_x, train_y = dfX.iloc[train_index], y.iloc[train_index]
        test_x, test_y = dfX.iloc[test_index], y.iloc[test_index]

        train_pool = Pool(data=train_x, label=train_y)
        test_pool = Pool(data=test_x, label=test_y)

        model.fit(train_pool, verbose=False)
        pred = model.predict(test_pool)
        preds.append(pred)
        vals.append(test_y) 

    results = utils.error_rate("Catboost",flatten(preds),flatten(vals))
    print(results)
    rmse = math.sqrt(mean_squared_error(flatten(vals),flatten(preds)))
    return rmse


sampler = optuna.samplers.TPESampler(seed=1)
study = optuna.create_study(direction="minimize",sampler = sampler)

study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2023-06-22 17:46:42,406] A new study created in memory with name: no-name-1520e0f2-acbb-4aca-bd9d-6f27a953792b
[I 2023-06-22 17:47:14,076] Trial 0 finished with value: 0.0347181719704004 and parameters: {'learning_rate': 0.015811390404929584, 'depth': 6, 'l2_leaf_reg': 1.0006061777325413e-08, 'bootstrap_type': 'Bayesian', 'random_strength': 8.558330239225555, 'bagging_temperature': 3.2935117816342263, 'od_type': 'Iter', 'od_wait': 41}. Best is trial 0 with value: 0.0347181719704004.


('Catboost', 0.0347181719704004, 0.028117088151993257, 0.4877369058741262)


[I 2023-06-22 17:47:39,245] Trial 1 finished with value: 0.03411433046248605 and parameters: {'learning_rate': 0.012270682125200601, 'depth': 5, 'l2_leaf_reg': 8.184155256441033e-08, 'bootstrap_type': 'Bayesian', 'random_strength': 9.022081038012919, 'bagging_temperature': 3.8383890288065894, 'od_type': 'Iter', 'od_wait': 47}. Best is trial 1 with value: 0.03411433046248605.


('Catboost', 0.03411433046248605, 0.027595848216313324, 0.5054011838367249)


[I 2023-06-22 17:48:11,291] Trial 2 finished with value: 0.03474061829250535 and parameters: {'learning_rate': 0.012518391049556016, 'depth': 6, 'l2_leaf_reg': 1.1561646150431216e-08, 'bootstrap_type': 'Bayesian', 'random_strength': 9.291054383008401, 'bagging_temperature': 3.834609604734254, 'od_type': 'Iter', 'od_wait': 46}. Best is trial 1 with value: 0.03411433046248605.


...

('Catboost', 0.03394110480238249, 0.027808720349181058, 0.510411376298824)


[I 2023-06-22 17:54:50,445] Trial 18 finished with value: 0.03391621743763539 and parameters: {'learning_rate': 0.010055565569171513, 'depth': 4, 'l2_leaf_reg': 9.05387537772107e-07, 'bootstrap_type': 'Bayesian', 'random_strength': 9.624745113704193, 'bagging_temperature': 4.0408910776767195, 'od_type': 'Iter', 'od_wait': 50}. Best is trial 18 with value: 0.03391621743763539.


('Catboost', 0.03391621743763539, 0.027800376526824943, 0.511129096216774)


[I 2023-06-22 17:55:11,840] Trial 19 finished with value: 0.03431404308759043 and parameters: {'learning_rate': 0.015320993175415047, 'depth': 4, 'l2_leaf_reg': 1.9536113104252325e-06, 'bootstrap_type': 'Bayesian', 'random_strength': 9.21060946152013, 'bagging_temperature': 4.024225473030197, 'od_type': 'Iter', 'od_wait': 50}. Best is trial 18 with value: 0.03391621743763539.


('Catboost', 0.03431404308759043, 0.028142560935429186, 0.4995932574115517)
Number of finished trials: 20
Best trial:
  Value: 0.03391621743763539
  Params: 
    learning_rate: 0.010055565569171513
    depth: 4
    l2_leaf_reg: 9.05387537772107e-07
    bootstrap_type: Bayesian
    random_strength: 9.624745113704193
    bagging_temperature: 4.0408910776767195
    od_type: Iter
    od_wait: 50


In [51]:
trialsdf2  = study.trials_dataframe(multi_index=True)
trialsdf2['number'] = trialsdf2['number'] + 1
trialsdf2.to_excel('optuna2.xlsx')

In [54]:
res = cbpred_params(dfX[cols],y,kf, trial.params)
res

{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'od_pval': 0, 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 9.053875373865594e-07, 'random_strength': 9.62474536895752, 'od_type': 'Iter', 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'use_best_model': False, 'od_wait': 50, 'random_seed': 0, 'depth': 4, 'posterior_sampling': False, 'border_count': 254, 'bagging_temperature': 4.040891170501709, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE'

('Catboost', 0.03391621743763539, 0.027800376526824943, 0.511129096216774)

In [52]:
relief = {'Indeks Pembangunan Manusia': 0.13284091911102613,
 'SD': 0.12034276029063282,
 'pendidikan_SMA': 0.11226924394021717,
 'pendidikan_SD': 0.09862159114928643,
 'DPT': 0.08951284040805399,
 'Persentase Kemiskinan': 0.08670741003840189,
 'pendidikan_Universitas': 0.0770181753771101,
 'Pengeluaran Bulanan': 0.06752418168824369,
 'Literate': 0.058178733750543235,
 'Perempuan': 0.052779796704443833,
 'Laki': 0.04879084066123379,
 'TK': 0.04016274053710977,
 'TPAK': 0.03947882612619671,
 'Laju Pertumbuhan': 0.03707727177524649,
 'Umur': 0.03485968154348226,
 'Umur Squared': 0.03442564055727986,
 'SMK': 0.033480552990718025,
 'SMP': 0.02936339912835733,
 'Sentiment': 0.017851810718429697,
 'Weighted Sentiment': 0.017851810718429697,
 'APM SD': 0.015954416625980503,
 'SMA': 0.014888048240001402,
 'pendidikan_SMP': 0.013984129438205239,
 'APM SMP': 0.013054998632209899,
 'APM SMA': 0.012096499861815853,
 'Universitas': -0.013497161469518246}

relief = pd.DataFrame.from_dict(relief, orient='index', columns=['ReliefF'])
relief.sort_values(by='ReliefF', ascending=False, inplace=True)
relief.to_excel('relief.xlsx')